In [1]:
import tensorflow as tf;import tensorflow.keras as keras;import numpy as np

In [2]:
t=tf.constant([[1,2,3],[4,5,6]])

In [3]:
t[:,1][:,np.newaxis]

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [5]], dtype=int32)>

# Customizing Models and Training Algorithms

### Custom Loss Functions 

In [4]:
def huber_loss(y_label,y_pred):
  error=y_label-y_pred
  is_small_erro= tf.abs(error)<1
  squared_loss= tf.square(error)/2
  linear_loss= tf.abs(error)-0.5
  return tf.where(is_small_erro,squared_loss,linear_loss)
  

In [5]:
from sklearn.datasets import  fetch_california_housing

(data,target)= fetch_california_housing(return_X_y=True)



In [7]:
from sklearn.model_selection import train_test_split;import numpy as np;from sklearn.preprocessing import StandardScaler
X_train_full,X_test,y_train_full,y_test= train_test_split(data,target[:,np.newaxis],random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)



In [8]:
class HuberLoss(keras.losses.Loss):

    def __init__(self,threshold=1,**kwargs):
        self.threshold=threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error=y_true-y_pred
        is_small_error= tf.abs(error)<self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config= super().get_config()
        return {**base_config,"threshold":self.threshold}

In [16]:
modelo= keras.models.Sequential([keras.layers.Dense(30,activation='selu',kernel_initializer='lecun_normal',input_shape=X_train.shape[1:]),
                                 keras.layers.Dense(1)])

In [17]:
modelo.compile(optimizer='nadam',loss=keras.losses.mean_squared_error,metrics=['mae'])

In [18]:
modelo.fit(x=X_train,y=y_train,validation_data=(X_valid,y_valid),epochs=50)

Epoch 1/50
363/363 [==============================] - 1s 2ms/step - loss: 24762.7598 - mae: 50.6359 - val_loss: 304.5815 - val_mae: 3.3856
Epoch 2/50
363/363 [==============================] - 1s 2ms/step - loss: 14.4230 - mae: 2.5995 - val_loss: 260.9261 - val_mae: 2.7610
Epoch 3/50
363/363 [==============================] - 1s 2ms/step - loss: 11.1747 - mae: 2.2288 - val_loss: 227.3809 - val_mae: 2.4548
Epoch 4/50
363/363 [==============================] - 1s 2ms/step - loss: 9.1452 - mae: 1.9925 - val_loss: 194.3756 - val_mae: 2.1403
Epoch 5/50
363/363 [==============================] - 1s 2ms/step - loss: 7.3678 - mae: 1.7567 - val_loss: 163.1570 - val_mae: 1.9363
Epoch 6/50
363/363 [==============================] - 1s 2ms/step - loss: 5.8638 - mae: 1.5419 - val_loss: 134.8964 - val_mae: 1.6872
Epoch 7/50
363/363 [==============================] - 1s 2ms/step - loss: 4.7268 - mae: 1.3767 - val_loss: 105.8088 - val_mae: 1.4887
Epoch 8/50
363/363 [==============================] - 1

### Custom Layers